# Pricing Strategy Development
Follow this step-by-step process for recommendations on Anchors, Benchmarks and Target Price Indexes for optimized sales or profit.

- Step 0: Select Segment, Market & Channel
- Step 1: Identify Anchors
- Step 2: Identify Pairs
- Step 3: See results

In [1]:
%%capture
# Libraries
!pip install pythonnet
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install datetime
!pip install scikit-learn
!pip install voila
!pip install ipywidgets

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:
import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning, message="divide by zero encountered")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="invalid value encountered")
warnings.filterwarnings("ignore", category=RuntimeWarning, message="Degrees of freedom <= 0 for slice")
warnings.filterwarnings("ignore", category=UserWarning)

In [20]:
# Connect to database

current_dir = os.getcwd()
file_path = os.path.join(current_dir, 'retail_price.csv')
retail_price = pd.read_csv(file_path)
# retail_price

# Include value share and volume share data into the database
categ_totals = retail_price.groupby(['product_category_name'])[['total_price', 'qty']].sum().reset_index()
retail_price = retail_price.merge(categ_totals, left_on='product_category_name', right_on='product_category_name', suffixes=('', '_categ_ttl'))
retail_price['val_share'] = retail_price['total_price'] / retail_price['total_price_categ_ttl']
retail_price['vol_share'] = retail_price['qty'] / retail_price['qty_categ_ttl']
# retail_price

,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,ps2,fp2,comp_3,ps3,fp3,lag_price,total_price_categ_ttl,qty_categ_ttl,val_share,vol_share
0,bed1,bed_bath_table,01-05-2017,1,45.95,15.100000,45.95,39,161,2,...,4.4,8.760000,45.950000,4.0,15.100000,45.900000,95084.52,1026,0.000483,0.000975
1,bed1,bed_bath_table,01-06-2017,3,137.85,12.933333,45.95,39,161,2,...,4.4,21.322000,45.950000,4.0,12.933333,45.950000,95084.52,1026,0.001450,0.002924
2,bed1,bed_bath_table,01-07-2017,6,275.70,14.840000,45.95,39,161,2,...,4.4,22.195932,45.950000,4.0,14.840000,45.950000,95084.52,1026,0.002900,0.005848
3,bed1,bed_bath_table,01-08-2017,4,183.80,14.287500,45.95,39,161,2,...,4.4,19.412885,45.950000,4.0,14.287500,45.950000,95084.52,1026,0.001933,0.003899
4,bed1,bed_bath_table,01-09-2017,2,91.90,15.100000,45.95,39,161,2,...,4.4,24.324687,45.950000,4.0,15.100000,45.950000,95084.52,1026,0.000967,0.001949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,furniture1,furniture_decor,01-04-2018,17,595.00,13.469412,35.00,49,829,1,...,3.7,13.469412,99.900000,4.4,23.327692,35.485714,56925.16,894,0.010452,0.019016
672,furniture1,furniture_decor,01-05-2018,11,385.00,14.634545,35.00,49,829,1,...,3.7,14.634545,99.900000,4.4,19.410769,35.000000,56925.16,894,0.006763,0.012304
673,furniture1,furniture_decor,01-06-2018,13,455.00,18.101538,35.00,49,829,1,...,3.7,18.101538,99.900000,4.4,25.636000,35.000000,56925.16,894,0.007993,0.014541
674,furniture1,furniture_decor,01-07-2018,10,350.00,17.238000,35.00,49,829,1,...,3.7,17.238000,78.535714,4.4,23.847143,35.000000,56925.16,894,0.006148,0.011186


In [5]:
# retail_price.columns

In [16]:
# Keep only useful columns

retail_price = retail_price[['product_id', 'product_category_name', 'month_year', 'qty',
       'total_price', 'unit_price']]
# retail_price

In [17]:
# Financials data (using dummy financials for confidentiality purposes)
data = {
    'PRODUCT NAME': ['bed2', 'watches6', 'furniture2'],
    'LP per IT': [80, 110, 60],
    'NOS per SU': [75, 100, 50],
    'TDC per SU': [30, 55, 40]
}
financials = pd.DataFrame(data)

In [7]:
# Function to identify anchors: receives sales by sku dataframe and returs list of top-10 selling skus

def identify_anchors(df):
    df = df.groupby('product')[['sales']].sum().sort_values('sales', ascending=False)

    limit = df['sales'].sum()*0.001
    df_limited = df[df['sales'] > limit]
    
    plt.figure(figsize=(12,2))
    
    plt.bar(df_limited.index, df_limited['sales'], color='blue')
    plt.title('Sales by sku', fontsize=12)
    plt.xticks(rotation=90, fontsize=6);
    plt.show()

    df_top_10 = df.head(10)
    perc = df_top_10['sales'].sum() / df['sales'].sum()

    print(f"Top 10 represents: {round(perc,2)*100}%")

    return df_top_10.index.to_list()

In [8]:
# Function to get correlations between anchor and other skus, to idenfity best benchmark

def get_correlation(df, product, all_products):
    df['price'] = df['sales_mlc'] / df['sales_msu']

    # Filter all_products database with same category as anchor
    category = all_products[all_products['product'] == product]['manufacturer'].values[0]
    same_categ_products = all_products[all_products['manufacturer'] == category]

    correlations = pd.DataFrame()    # initiate correlations df
    df1 = df[df['product'] == product]
    sales_mlc_anchor = df1['sales_mlc'].sum()

    for i in range(len(same_categ_products)):
        df2 = df[df['product'] == same_categ_products['product'].values[i]]
        merged_df = df1.merge(df2, left_on='date', right_on='date', suffixes=('_anchor', '_pair'))
        merged_df['price_ix'] = merged_df['price_anchor'] / merged_df['price_pair']
        merged_df[['price_ix', 'val_share_anchor', 'vol_share_anchor']].corr()
        ix_val_share_corr = merged_df['price_ix'].corr(merged_df['val_share_anchor'])
        ix_vol_share_corr = merged_df['price_ix'].corr(merged_df['vol_share_anchor'])
        corr_2 = pd.DataFrame()

        sales_mlc_pair = df2['sales_mlc'].sum()
        
        data = {'product_anchor': [product],
                'product_pair': [same_categ_products['product'].values[i]],
                'ix_val_share_corr': [ix_val_share_corr],
                'ix_vol_share_corr': [ix_vol_share_corr],
                'val_share_anchor': [df1['val_share'].mean()],
                'val_share_pair': [df2['val_share'].mean()],
                'price_anchor': [df1['price'].mean()],
                'price_pair': [df2['price'].mean()],
                'val_share_ix': [round(df1['val_share'].mean() / df2['val_share'].mean(),2)],
                'price_ix': [round(df1['price'].mean() / df2['price'].mean(),2)],
               }
        # columns = ['product_anchor', 'product_pair', 'ix_val_share_corr', 'ix_vol_share_corr', '', '', '', '']
        corr_2 = pd.DataFrame(data)
        correlations = pd.concat([correlations, corr_2])

        correlations.dropna(inplace=True)
#         correlations = correlations[(correlations['price_ix'] > 0.5) & (correlations['price_ix'] < 2)]
#         correlations = correlations[(correlations['val_share_ix'] > 0.1) & (correlations['val_share_ix'] < 40)]
#         correlations = correlations[(correlations['ix_val_share_corr'] < -0.3) & (correlations['ix_val_share_corr'] < -0.3)]
        correlations.sort_values('ix_vol_share_corr', inplace=True)
        
    return correlations

In [9]:
# Function to evaluate pair of anchor-benchmark

def evaluate_pairs(df, anchor, pair):
    df1 = df[df['product'] == anchor]
    df2 = df[df['product'] == pair]
    merged_df = df1.merge(df2, left_on='date', right_on='date', suffixes=('_anchor', '_pair'))
    
    merged_df['price_per_it_anchor'] = merged_df['price_anchor'] / (36 / merged_df['count_anchor'].astype(float))
    merged_df['price_per_it_pair'] = merged_df['price_pair'] / (36 / merged_df['count_pair'].astype(float))
    merged_df['index'] = (merged_df['price_per_it_anchor'] / merged_df['price_per_it_pair'])*100
    merged_df.sort_values('date', inplace=True)

    # Create a 3x1 grid of subplots
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12, 3))
    
    ax = axes[0]
    
    ax.plot(merged_df['date'], merged_df['price_per_it_anchor'], label='Anchor', color='blue', alpha=0.8)
    ax.plot(merged_df['date'], merged_df['price_per_it_pair'], label='Pair', color='red', alpha=0.8)
    ax.set_ylabel('Price LC/IT')
    ax.tick_params(axis='x', rotation=70)
    ax.legend(loc='upper left')
    
    ax2 = ax.twinx()
    ax2.plot(merged_df['date'], merged_df['index'], label='Index',  linestyle='dotted', color='grey', alpha=0.8)
    ax2.set_ylabel('Index')
    ax2.legend(loc='lower right')
    
    ax.set_title('Anchor vs Pair prices & index')
    
    
    ax = axes[1]
    
    ax.plot(merged_df['date'], merged_df['val_share_anchor'], label='$ shares', color='lightblue', alpha=0.8)
    ax.plot(merged_df['date'], merged_df['vol_share_anchor'], label='Vol shares', color='green', alpha=0.8)
    ax.set_ylabel('Shares')
    ax.tick_params(axis='x', rotation=70)
    ax.legend(loc='upper left')
    
    ax2 = ax.twinx()
    ax2.plot(merged_df['date'], merged_df['index'], label='Index', linestyle='dotted', color='grey', alpha=0.8)
    ax2.set_ylabel('Index')
    ax2.legend(loc='lower right')
    
    ax.set_title('Anchor shares & index vs pair')
    
    
    ax = axes[2]
    ax.scatter(merged_df['index'], merged_df['vol_share_anchor'] , marker='.')
    
    # Fit a linear regression line
    coefficients = np.polyfit(merged_df['index'], merged_df['vol_share_anchor'], 1)
    trendline = np.poly1d(coefficients)
    x = np.linspace(min(merged_df['index']), max(merged_df['index']), 100)
    ax.plot(x, trendline(x), color='grey', linestyle='dotted', label='Trendline', alpha=0.8)
    
    ax.set_ylabel('Vol shares')
    ax.set_xlabel('price index')
    ax.set_title('Vol Share vs Price Index')
    
    plt.tight_layout()
    plt.show()
    
    return merged_df

In [10]:
# Function to get linear model for shares per price index, to identify optimized price index

def get_linear_model(df):

    X = df[['index']]
    y = df['vol_share_anchor']

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=41)

    model = LinearRegression()

    model.fit(X_train, y_train)
    # print(model.score(X_test, y_test))
    
    plt.figure(figsize=(4,2))

    y_predicted = model.predict(df[['index']])

    plt.plot(df['index'], y_predicted, color='blue', label='Predicted')
    plt.scatter(df['index'], df['vol_share_anchor'], marker='.', color='red', label='Actual')
    plt.legend()

    plt.ylabel('vol shares')
    plt.xlabel('price index')
    plt.title('Vol Share vs Price Index')
    plt.show()

    return model

In [24]:
# Function to identify optimal price index

def find_optimal_ix(financials, merged_df, anchor):
    anchor_base_lp = financials[financials['PRODUCT NAME'] == anchor.strip()]['LP per IT'].values[0]
    anchor_nos_per_su = financials[financials['PRODUCT NAME'] == anchor.strip()]['NOS per SU'].values[0]
    anchor_tdc_per_su = financials[financials['PRODUCT NAME'] == anchor.strip()]['TDC per SU'].values[0]
    # print(f'list price lc per it {anchor_base_lp}')
    # print(f'nos $ per su {anchor_nos_per_su}')
    # print(f'tdc $ per su {anchor_tdc_per_su}')

    # min = round(merged_df['index'].min()/10,0)*10
    # max = round(merged_df['index'].max()/10,0)*10
    min = 10
    max = 200
    # outer_range = (max - min)/2
    # tick = ((max + outer_range) - (min - outer_range))/5
    # tick = tick.astype(int)
    index_scenarios = [(min)]
    for i in range(34):
        index_scenarios.append((min) + (i+1)*5)

    merged_df_period = merged_df

    anchor_base_price_point = (merged_df_period['sales_mlc_anchor'].sum() / merged_df_period['sales_msu_anchor'].sum()) / \
                (36 / int(merged_df['count_anchor'][0]))
    pair_base_price_point = (merged_df_period['sales_mlc_pair'].sum() / merged_df_period['sales_msu_pair'].sum()) / \
                (36 / int(merged_df['count_pair'][0]))
    anchor_base_vol_share = model.predict([[(anchor_base_price_point / pair_base_price_point)*100]])[0]

    merged_df_year = merged_df

    anchor_yr_vol = merged_df_year['sales_msu_anchor'].sum()

    # print(f'index scenarios {index_scenarios}')
    # print(f'anchor vol share {anchor_base_vol_share}')
    # print(f'anchor year volume {anchor_yr_vol}')
    # print(f'anchor price {anchor_base_price_point}')
    # print(f'pair price {pair_base_price_point}')

    scenarios_df = pd.DataFrame(index=index_scenarios)
    
    for i, index in enumerate(index_scenarios):
        scenarios_df.at[index, 'price_point_lc'] = round(pair_base_price_point * index / 100, 2)
        scenarios_df.at[index, 'gs_lc_per_it'] = round(anchor_base_lp *
                                                       scenarios_df.at[index, 'price_point_lc'] /
                                                       anchor_base_price_point, 2)
        scenarios_df.at[index, 'vol_share'] = model.predict([[index]])[0]
        scenarios_df.at[index, 'incr_vol'] = (anchor_yr_vol * scenarios_df.at[index, 'vol_share'] / anchor_base_vol_share) - \
                                        anchor_yr_vol
        
    #         print(f'index: {index}')
    #         print(f'pair_sku_current_price_point: {pair_sku_current_price_point}')
    #         print(f'base_sku_yr_msu: {base_sku_yr_msu}')
    #         print(f'base_sku_vol_share: {base_sku_vol_share}')
    #         print(f"scenario vol share: {scenarios_df.at[index, 'vol_share']}")
        
        scenarios_df.at[index, 'nos_per_su'] = anchor_nos_per_su * \
                                        scenarios_df.at[index, 'price_point_lc'] / \
                                        anchor_base_price_point
        scenarios_df.at[index, 'incr_nos'] = (anchor_yr_vol * scenarios_df.at[index, 'vol_share'] / anchor_base_vol_share) * \
                                        scenarios_df.at[index, 'nos_per_su'] - \
                                        (anchor_yr_vol * anchor_nos_per_su)
        scenarios_df.at[index, 'gm_%'] = '{:.2%}'.format((scenarios_df.at[index, 'nos_per_su'] - anchor_tdc_per_su) /
                                        scenarios_df.at[index, 'nos_per_su'])
        scenarios_df.at[index, 'incr_gm'] = ((scenarios_df.at[index, 'nos_per_su'] - anchor_tdc_per_su) *
                                        (anchor_yr_vol * scenarios_df.at[index, 'vol_share'] / anchor_base_vol_share)) - \
                                        ((anchor_nos_per_su - anchor_tdc_per_su) * 
                                        anchor_yr_vol)
    
    fig, ax1 = plt.subplots(figsize=(4, 2))
    
    ax1.plot(scenarios_df.index, scenarios_df['incr_nos'], label='incr nos', color='blue', alpha=0.8)
    ax1.plot(scenarios_df.index, scenarios_df['incr_gm'], label='incr gc', color='green', alpha=0.8)
    
    ax1.set_ylabel('$M')
    ax1.legend(loc='lower left')
    
    ax2 = ax1.twinx()
    ax2.plot(scenarios_df.index, scenarios_df['incr_vol'], label='incr vol', color='black', alpha=0.8)
    ax2.set_ylabel('MSU')
    ax2.legend(loc='upper right')
    
    max_incr_nos = scenarios_df['incr_nos'].max()
    max_incr_gm = scenarios_df['incr_gm'].max()
    
    max_incr_nos_index = scenarios_df.index[scenarios_df['incr_nos'] == max_incr_nos].tolist()
    max_incr_gm_index = scenarios_df.index[scenarios_df['incr_gm'] == max_incr_gm].tolist()
    
    print(f"Max incr_nos: {max_incr_nos:.2f}, Index: {max_incr_nos_index}")
    print(f"Max incr_gc: {max_incr_gm:.2f}, Index: {max_incr_gm_index}")
    
    highlight_max_nos = max_incr_nos_index
    highlight_max_gm = max_incr_gm_index
    
    ax1.axvline(x=highlight_max_nos, color='blue', linestyle='dotted', alpha=0.3)
    ax1.axvline(x=highlight_max_gm, color='green', linestyle='dotted', alpha=0.3)
    
    # # Set y-axis limits symmetrically around zero
    # y1_min, y1_max = ax1.get_ylim()
    # y1_lim = max(abs(y1_min), abs(y1_max))
    # ax1.set_ylim(-y1_lim, y1_lim)
    
    # y2_min, y2_max = ax2.get_ylim()
    # y2_lim = max(abs(y2_min), abs(y2_max))
    # ax2.set_ylim(-y2_lim, y2_lim)
    
    # Add horizontal line at zero position on y-axis
    ax1.axhline(y=0, color='black', linestyle='dotted', alpha=0.1)
    
    plt.title('Incr NOS, GC, Vol scenarios')
    plt.show()
    
    display(scenarios_df)

    return scenarios_df

In [26]:
# Function triggered by first button: prompt selection of Category
# and prepare options for second selection

def seg_mkt_chan_button_clicked(b):
    global sales_by_sku_df
    global filtered_df
    global all_products
    global anchors
    
    # Output widget
    with out:
        clear_output()
        
        # Get user selection
        segment = segment_dropdown.value
        
        # Sales by sku dataframe
        sales_by_sku_df = retail_price[retail_price['product_category_name'] == segment]
        sales_by_sku_df = sales_by_sku_df[['product_id', 'total_price']]
        sales_by_sku_df.rename(columns={'product_id': 'product',
                                                  'total_price': 'sales'}, inplace=True)

        # filtered dataframe
        filtered_df = retail_price
        filtered_df['count'] = 1
        filtered_df = filtered_df.rename(columns={'month_year': 'date',
                                                  'product_category_name': 'manufacturer',
                                                  'product_id': 'product',
                                                  'total_price': 'sales_mlc',
                                                  'qty': 'sales_msu'})
        filtered_df = filtered_df.groupby(['date', 'manufacturer', 'count',
            'product'])[['sales_msu', 'sales_mlc', 'val_share', 'vol_share']].sum().reset_index()

        # All products dataframe
        all_products = filtered_df.groupby(['product', 'manufacturer'])[['sales_mlc']].sum().sort_values('sales_mlc', ascending=False)
        limit = all_products['sales_mlc'].sum()*0.001
        all_products_limited = all_products[all_products['sales_mlc'] > limit]
        print(all_products_limited['sales_mlc'].sum() / all_products['sales_mlc'].sum())
        all_products = all_products.reset_index()[['product', 'manufacturer']]

        # Call identify anchors function
        anchors = identify_anchors(sales_by_sku_df)
        
        # Update anchors options list
        anchor_dropdown.options=anchors
        
        # Display the Markdown string
        display(Markdown("# Select anchor to analyze: \n Restart here for each anchor to be analyzed. \n For this summarized exercise, anchors should be one of the following (no dummy financial data for the rest): 'bed2', 'watches6', 'furniture2'"))

In [22]:
# Function triggered by second button: prompt selection of Anchor
# and prepare options for third selection

def anchors_button_clicked(b):
    global anchor
    
    # Output widget
    with out2:
        clear_output()
        
        # Get user selection
        anchor = anchor_dropdown.value
        print(f"Selected anchor: {anchor}")
        
        # Display Markdown string
        display(Markdown("# Select Benchmark: "))
        
        # Call get_correlation function
        correlations = get_correlation(filtered_df, anchor, all_products)
        display(correlations)
        
        # Update benchmarks options list
        benchmark_dropdown.options = correlations['product_pair']

In [15]:
# Function triggered by third button: prompt selection of Benchmark

def benchmark_button_clicked(b):
    global model
    
    with out3:
        clear_output()
        
        # Get user selection
        pair = benchmark_dropdown.value
        print(f"Selected anchor: {anchor}")
        print(f"Selected benchmark: {pair}")
        
        # Call evaluate_pairs function
        merged_df = evaluate_pairs(filtered_df, anchor, pair)
        
        # Call get_linear_model function
        model = get_linear_model(merged_df)
        
        # Call find_optimal_ix function
        find_optimal_ix(financials, merged_df, anchor)

# Select Category

In [27]:
style = {'description_width': 'initial'}

# Dropdowns
segment_options = retail_price['product_category_name'].unique()
segment_dropdown = widgets.Dropdown(
    options=segment_options,
    value=segment_options[0],
    description='Choose Category',
    style=style)

anchor_dropdown = widgets.Dropdown(
    options=[],
    description='Choose Anchor',
    style=style)

benchmark_dropdown = widgets.Dropdown(
    options=[],
    description='Choose Benchmark',
    style=style)

# Output widgets
out = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

# Buttons
seg_mkt_chan_button = widgets.Button(description='Click after selection')
anchors_button = widgets.Button(description='Click after selection')
benchmark_button = widgets.Button(description='Click after selection')

# Attach the function to the button's on_click event
seg_mkt_chan_button.on_click(seg_mkt_chan_button_clicked)
anchors_button.on_click(anchors_button_clicked)
benchmark_button.on_click(benchmark_button_clicked)

# Display the widgets
display(segment_dropdown, seg_mkt_chan_button, out)
display(anchor_dropdown, anchors_button, out2)
display(benchmark_dropdown, benchmark_button, out3)

Dropdown(description='Choose Category', options=('bed_bath_table', 'garden_tools', 'consoles_games', 'health_b…

Button(description='Click after selection', style=ButtonStyle())

Output()

Dropdown(description='Choose Anchor', options=(), style=DescriptionStyle(description_width='initial'), value=N…

Button(description='Click after selection', style=ButtonStyle())

Output()

Dropdown(description='Choose Benchmark', options=(), style=DescriptionStyle(description_width='initial'), valu…

Button(description='Click after selection', style=ButtonStyle())

Output()